In [2]:
import cv2
import imutils
import numpy as np

cap = cv2.VideoCapture('test-video.mp4')
fgbg = cv2.createBackgroundSubtractorMOG2()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
car_counter = 0

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame = imutils.resize(frame, width=640)

    area_pts = np.array([[60, 196], [frame.shape[1] - 145, 196], [frame.shape[1] - 60, 240], [140, 240]])

    imAux = np.zeros(shape=(frame.shape[:2]), dtype=np.uint8)
    imAux = cv2.drawContours(imAux, [area_pts], -1, (255), -1)
    image_area = cv2.bitwise_and(frame, frame, mask=imAux)

    fgmask = fgbg.apply(image_area)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    fgmask = cv2.dilate(fgmask, None, iterations=5)

    cnts = cv2.findContours(fgmask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]

    detections = []

    for cnt in cnts:
        if cv2.contourArea(cnt) > 500:
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append([x, y, w, h])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
            if 216 < (y + h) < 219:
                car_counter = car_counter + 1

    cv2.rectangle(frame, (frame.shape[1] - 70, 215), (frame.shape[1] - 5, 270), (0, 255, 0), 2)
    cv2.putText(frame, str(car_counter), (frame.shape[1] - 55, 250),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
    cv2.imshow('detector', frame)
    cv2.imshow('bn', fgmask)

    k = cv2.waitKey(15) & 0xFF
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()
print(car_counter)

12
